<a href="https://colab.research.google.com/github/abdirimoof/ML/blob/main/HuggingFace_5amaliy_vazifa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from transformers import AutoTokenizer

# Load the DistilBERT tokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [4]:
def preprocess_function(examples):
    return tokenizer(examples['sentence'], truncation=True, padding=True, max_length=128)

# Apply the preprocessing function to the datasets
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_validation_dataset = validation_dataset.map(preprocess_function, batched=True)

# Display a sample of the tokenized dataset
print("Sample from tokenized_train_dataset:")
print(tokenized_train_dataset[0])

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Sample from tokenized_train_dataset:
{'idx': 32326, 'sentence': 'klein , charming in comedies like american pie and dead-on in election , ', 'label': 1, 'input_ids': [101, 12555, 1010, 11951, 1999, 22092, 2066, 2137, 11345, 1998, 2757, 1011, 2006, 1999, 2602, 1010, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [3]:
import datasets

# Load the SST-2 dataset
dataset = datasets.load_dataset('sst2')

# Extract smaller subsets
train_dataset = dataset['train'].shuffle(seed=42).select(range(2000))
validation_dataset = dataset['validation'].shuffle(seed=42).select(range(400))

# Display information about the new subsets
print(f"Original training set size: {len(dataset['train'])}")
print(f"New training set size: {len(train_dataset)}")
print(f"Original validation set size: {len(dataset['validation'])}")
print(f"New validation set size: {len(validation_dataset)}")

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Original training set size: 67349
New training set size: 2000
Original validation set size: 872
New validation set size: 400


In [5]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# Load the DistilBERT model for sequence classification
# The SST-2 dataset has 2 labels (positive/negative)
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2, # As requested
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=50, # As requested
    evaluation_strategy="epoch", # Re-added for proper evaluation
    report_to="none" # Disable logging to wandb to avoid interactive prompt
)

In [11]:
# Create a Trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_validation_dataset
)

# Train the model
trainer.train()

Step,Training Loss
50,0.517600
100,0.329500
150,0.270700
200,0.183500
250,0.178900


TrainOutput(global_step=250, training_loss=0.2960732231140137, metrics={'train_runtime': 33.4468, 'train_samples_per_second': 119.593, 'train_steps_per_second': 7.475, 'total_flos': 54849782256000.0, 'train_loss': 0.2960732231140137, 'epoch': 2.0})

In [12]:
# Evaluate the model
eval_results = trainer.evaluate(eval_dataset=tokenized_validation_dataset)
print(f"Evaluation results: {eval_results}")
print(f"Evaluation Loss: {eval_results['eval_loss']:.4f}")

Evaluation results: {'eval_loss': 0.40914779901504517, 'eval_runtime': 0.8969, 'eval_samples_per_second': 445.996, 'eval_steps_per_second': 27.875, 'epoch': 2.0}
Evaluation Loss: 0.4091


Yangi matnlar bilan sinovdan o'tkazish

In [13]:
import torch

# New texts to test
new_texts = [
    "this movie is a masterpiece",
    "a complete waste of time"
]

# Tokenize the new texts
inputs = tokenizer(new_texts, truncation=True, padding=True, max_length=128, return_tensors="pt")

# Move inputs to the same device as the model
inputs = inputs.to(model.device)

# Get predictions (logits)
with torch.no_grad():
    outputs = model(**inputs)

# Apply softmax to get probabilities
predictions = torch.softmax(outputs.logits, dim=-1)

# Get predicted labels (0 or 1)
predicted_labels = torch.argmax(predictions, dim=-1).tolist()

# Define label mapping for SST-2 (0: negative, 1: positive)
label_map = {0: 'Negative', 1: 'Positive'}

# Display results
print("\n--- New Text Predictions ---")
for i, text in enumerate(new_texts):
    label = predicted_labels[i]
    prob = predictions[i][label].item() * 100
    all_probs = predictions[i].tolist()
    print(f"Text: '{text}'")
    print(f"  Predicted Label: {label_map[label]} (Label ID: {label})")
    print(f"  Probability of predicted label: {prob:.2f}%")
    print(f"  Probabilities for all classes (Neg, Pos): {all_probs[0]:.2f}, {all_probs[1]:.2f}\n")


--- New Text Predictions ---
Text: 'this movie is a masterpiece'
  Predicted Label: Positive (Label ID: 1)
  Probability of predicted label: 99.23%
  Probabilities for all classes (Neg, Pos): 0.01, 0.99

Text: 'a complete waste of time'
  Predicted Label: Negative (Label ID: 0)
  Probability of predicted label: 98.73%
  Probabilities for all classes (Neg, Pos): 0.99, 0.01

